# Exploratory Data Analysis (EDA) of a Pruvo dataset.

## Instructions:
Let me share with you a sample of the data we do have on our end and let me know what you think.

During the last years we were collecting a lot of human decisions regarding the relations of 2 room types.

In the following excel: https://docs.google.com/spreadsheets/d/1AjDaqnXrZFy3KAspjUAVo5cLt37MEVXOuBooYDM9Kr8/edit?usp=sharing

You can see the decisions made between the original room type (Col A) and the proposed one (Col B).

Links = how many times a person clicked that both rooms in A and B are the same (if it is higher than 6 it means someone put a "force" match decision – this is the admin 1000 score you see)

Unlinks = how many times someone decided it is not a match (different room type)

Score = links – unlinks + admin

I think we have a few millions of rows like that (of course that there are mistakes there as it was a human decision)

Let me know what you think…

## Initial Reflections
It seems that in the dataset I'm about to analyse there are 2 columns (A & B), A being the original, B being the one proposed by Pruvo, then a third column called "Links" that scores 